In [1]:
! pip install -q sacremoses

In [7]:
! mkdir cache

In [2]:
from datasets import load_dataset
from transformers import pipeline
import pandas as pd

# **Summarization**



In [6]:
from datasets import load_dataset

cnn_dataset = load_dataset(
    "cnn_dailymail",  '3.0.0' , cache_dir=None
)

print(cnn_dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})



* `article`: The full news article text.
* `highlights`: human-written highlights/summary of the article
* `id`: article ID

In [8]:
cnn_sample = cnn_dataset["train"].select(range(10))
display(cnn_sample.to_pandas())

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a
5,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili...",a1ebb8bb4d370a1fdf28769206d572be60642d70
6,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin...",7c0e61ac829a3b3b653e2e3e7536cc4881d1f264
7,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...,f0d73bdab711763e745cdc75850861c9018f235d
8,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can...",5e22bbfc7232418b8d2dd646b952e404df5bd048
9,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...,613d6311ec2c1985bd44707d1796d275452fe156


In [9]:
cnn_sample = cnn_dataset["train"].select(range(10))
display(cnn_sample.to_pandas())

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a
5,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili...",a1ebb8bb4d370a1fdf28769206d572be60642d70
6,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin...",7c0e61ac829a3b3b653e2e3e7536cc4881d1f264
7,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...,f0d73bdab711763e745cdc75850861c9018f235d
8,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can...",5e22bbfc7232418b8d2dd646b952e404df5bd048
9,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...,613d6311ec2c1985bd44707d1796d275452fe156


In [ ]:
summarizer = pipeline(
    task="summarization",
    model="t5-small",
    min_length=20,
    max_length=40,
    truncation=True,

)

In [12]:
# Apply to 1 article
summarizer(cnn_sample["article"][5])

[{'summary_text': 'new: youssif\'s father says he didn\'t know what to do with himself . new: "we knew there was kindness out there," he says . he\'s wearing a facial mask often used to help burn victims . the children\'s burn foundation agreed to pay for the transportation .'}]

In [56]:
# Apply to a batch of articles



results = summarizer(list(cnn_sample["article"]))

display(
    pd.DataFrame.from_dict(results)
    .rename({"summary_text": "generated_summary"}, axis=1)
    .join(pd.DataFrame.from_dict(cnn_sample))[
        ["generated_summary", "highlights", "article"]
    ]
)

,generated_summary,highlights,article
0,the young actor says he has no plans to fritte...,Harry Potter star Daniel Radcliffe gets £20M f...,"LONDON, England (Reuters) -- Harry Potter star..."
1,inmates with most severe mental illnesses are ...,Mentally ill inmates in Miami are housed on th...,Editor's note: In our Behind the Scenes series...
2,"survivor: ""i probably had a 30-, 35-foot free ...","NEW: ""I thought I was going to die,"" driver sa...","MINNEAPOLIS, Minnesota (CNN) -- Drivers who we..."
3,the polyps were removed and sent to the nation...,"Five small polyps found during procedure; ""non...",WASHINGTON (CNN) -- Doctors removed five small...
4,"new: ""your admitted conduct was not only illeg...","NEW: NFL chief, Atlanta Falcons owner critical...",(CNN) -- The National Football League has ind...
5,new: youssif's father says he didn't know what...,"Parents beam with pride, can't stop from smili...","BAGHDAD, Iraq (CNN) -- Dressed in a Superman s..."
6,women are too afraid and ashamed to show their...,"Aid workers: Violence, increased cost of livin...","BAGHDAD, Iraq (CNN) -- The women are too afrai..."
7,fugitive from drug trafficking indictment kill...,Tomas Medina Caracas was a fugitive from a U.S...,"BOGOTA, Colombia (CNN) -- A key rebel commande..."
8,"new: president bush says he will ""sadly accept...","President Bush says Tony Snow ""will battle can...",WASHINGTON (CNN) -- White House press secretar...
9,the 20-year-old AT4 anti-tank rocket launcher ...,Empty anti-tank weapon turns up in front of Ne...,(CNN) -- Police and FBI agents are investigati...


# 2. Sentiment analysis

Data: poem sentiment dataset, which provides lines from poems tagged with sentiments negative 0, positive 1, no_impact 2, or mixed 3.
Model: fine-tuned version of BERT.


In [ ]:
poem_dataset = load_dataset(
    "poem_sentiment",


)

poem_sample = poem_dataset["train"].select(range(10))
display(poem_sample.to_pandas())

In [ ]:
sentiment_classifier = pipeline(
    task="text-classification",
    model="nickwong64/bert-base-uncased-poems-sentiment"

)

In [18]:
results = sentiment_classifier(list(poem_sample["verse_text"]))

# Join predictions with ground-truth data
joined_data = (
    pd.DataFrame.from_dict(results)
    .rename({"label": "predicted_label"}, axis=1)
    .join(pd.DataFrame.from_dict(poem_sample).rename({"label": "true_label"}, axis=1))
)

sentiment_labels = {0: "negative", 1: "positive", 2: "no_impact", 3: "mixed"}
joined_data = joined_data.replace({"true_label": sentiment_labels})

display(joined_data[["predicted_label", "true_label", "score", "verse_text"]])

,predicted_label,true_label,score,verse_text
0,positive,positive,0.996594,with pale blue berries. in these peaceful shad...
1,no_impact,no_impact,0.998741,"it flows so long as falls the rain,"
2,negative,negative,0.995966,"and that is why, the lonesome day,"
3,mixed,mixed,0.968735,"when i peruse the conquered fame of heroes, an..."
4,mixed,mixed,0.975967,of inward strife for truth and liberty.
5,mixed,mixed,0.966580,the red sword sealed their vows!
6,no_impact,no_impact,0.998639,and very venus of a pipe.
7,no_impact,no_impact,0.998611,"who the man, who, called a brother."
8,negative,negative,0.996557,"and so on. then a worthless gaud or two,"
9,no_impact,no_impact,0.998519,to hide the orb of truth--and every throne


# 3. Translation
Below, we use an English-to-French model.


In [ ]:
en_to_es_translation_pipeline = pipeline(
    task="translation",
    model="Helsinki-NLP/opus-mt-en-es",
    model_kwargs={"cache_dir": "../working/cache"}
)

en_to_es_translation_pipeline(
    "Existing, open-source (and proprietary) models can be used out-of-the-box for many applications."
)

In [ ]:
t5_small_pipeline = pipeline(
    task="text2text-generation",
    model="t5-small",
    max_length=50,
    model_kwargs={"cache_dir": "../working/cache"},
)

t5_small_pipeline(
    "translate English to French: Existing, open-source (and proprietary) models can be used out-of-the-box for many applications."
)

# 4. Zero-shot classification




In [ ]:
zero_shot_pipeline = pipeline(
    task="zero-shot-classification",
    model="cross-encoder/nli-deberta-v3-small"

)


def categorize_article(article: str) -> None:

    results = zero_shot_pipeline(
        article,
        candidate_labels=[
            "politics",
            "finance",
            "sports",
            "science and technology",
            "pop culture",
            "breaking news",
        ],
    )
    # Print the results nicely
    del results["sequence"]
    display(pd.DataFrame(results))

In [23]:
categorize_article(
    """
Jordan Alvarez struck the decisive goal deep into stoppage time, adding to earlier efforts by Mason Greenwood and Callum Hudson‑Odoi as City snatched a 3‑2 win in a pulsating derby.
United’s Marcus Rashford and Bruno Fernandes each found the net in a thrilling encounter that kept both sets of supporters on edge until the final whistle.
City’s superior squad depth became evident in the closing stages as manager Luis Romero’s side overcame a ragged first half to dominate possession after the break.
Home coach Romero handed a debut to exciting Brazilian midfielder Pedro Luiz, while star defender Lucas Mendes returned after a six‑month layoff. United, meanwhile, gave first starts of the season to academy graduate Liam Carter and veteran left‑back Rafael Silva.
United struck first when Rashford latched onto a loose ball in the 18th minute and fired low beyond keeper Ederson. City responded ten minutes later, Greenwood finishing calmly after a flowing move down the right.
A contentious moment arrived before the interval when Pedro Luiz was shown a yellow card for a late challenge, and Fernandes converted the resulting free‑kick to restore United’s lead at 2‑1.
After half‑time, City gradually turned the screw. Substitute Hudson‑Odoi brought them level in the 67th minute, cutting inside from the flank and curling a superb shot into the far corner.
Both sides pushed for a winner as tackles flew in and the atmosphere inside the stadium reached fever pitch. Deep into added time, a quick counterattack saw Mendes deliver a pinpoint cross for Alvarez, who volleyed home to spark wild celebrations from the home support.
United head coach Marco Silva admitted afterward:
“We’re devastated by the result. The players gave everything, but lapses in concentration at key moments cost us. Still, there are many positives in the way we played.”
"""
)

,labels,scores
0,sports,0.369003
1,breaking news,0.270523
2,pop culture,0.141178
3,politics,0.080684
4,science and technology,0.079014
5,finance,0.059598


In [24]:
categorize_article(
    """
NASA engineers have begun the final assembly phase of the Europa Clipper spacecraft, a mission designed to investigate the icy moon Europa and assess its potential to host life.
Set for launch in October 2026, the spacecraft will carry a suite of high‑resolution cameras, spectrometers, and ice‑penetrating radar to map Europa’s surface and search for subsurface oceans.
Technicians at the Jet Propulsion Laboratory in California say that months of component testing have gone smoothly, allowing them to integrate the main science instruments ahead of schedule.
Europa, one of Jupiter’s largest moons, has long intrigued scientists because of strong evidence suggesting a salty ocean lies beneath its frozen crust. The Clipper’s data could inform future missions that might attempt to land and drill through the ice.
Project manager Tracy Young said the mission represents a “major leap forward” in the search for habitable environments beyond Earth:
“Every day in the clean room we are reminded that this spacecraft will travel half a billion miles to answer a question humanity has asked for decades—could we find conditions suitable for life?”
NASA expects the spacecraft to conduct nearly 50 close flybys of Europa during its four‑year mission, transmitting unprecedented data back to Earth.
"""
)

,labels,scores
0,science and technology,0.371988
1,breaking news,0.279814
2,politics,0.110618
3,sports,0.092392
4,pop culture,0.083659
5,finance,0.061528


# 5. Few-shot learning


 * **Model**: gpt-neo-1.3B


In [ ]:
# We will limit the response length for our few-shot learning tasks.
few_shot_pipeline = pipeline(
    task="text-generation",
    model="EleutherAI/gpt-neo-1.3B",
    max_new_tokens=10

)

***Tip***: In the few-shot prompts below, we separate the examples with a special token "###" and use the same token to encourage the LLM to end its output after answering the query.  We will tell the pipeline to use that special token as the end-of-sequence (EOS) token below.


In [26]:
# Get the token ID for "###", which we will use as the EOS token below.
eos_token_id = few_shot_pipeline.tokenizer.encode("###")[0]

In [27]:
# Without any examples
results = few_shot_pipeline(
    """For each tweet, describe its sentiment:

[Tweet]: "This new music video was incredible"
[Sentiment]:""",
    eos_token_id=eos_token_id,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet, describe its sentiment:

[Tweet]: "This new music video was incredible"
[Sentiment]: "Very excited"
[Tweet]: "That


In [30]:
# With 1 example
results = few_shot_pipeline(
    """For each tweet, describe its sentiment:

[Tweet]: "This article is not good"
[Sentiment]: Negative
###
[Tweet]: "This new music video was incredible"
[Sentiment]:""",
    eos_token_id=eos_token_id,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet, describe its sentiment:

[Tweet]: "This article is not good"
[Sentiment]: Negative
###
[Tweet]: "This new music video was incredible"
[Sentiment]: Positive
###


In [32]:
# With 1 example for each sentiment
results = few_shot_pipeline(
    """For each tweet, describe its sentiment:

[Tweet]: "I hate it when my phone battery dies."
[Sentiment]: Negative
###
[Tweet]: "My day has been 👍"
[Sentiment]: Positive
###
[Tweet]: "This is the link to the article"
[Sentiment]: Neutral
###
[Tweet]: "This new music video was incredible"
[Sentiment]:""",
    eos_token_id=eos_token_id,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet, describe its sentiment:

[Tweet]: "I hate it when my phone battery dies."
[Sentiment]: Negative
###
[Tweet]: "My day has been 👍"
[Sentiment]: Positive
###
[Tweet]: "This is the link to the article"
[Sentiment]: Neutral
###
[Tweet]: "This new music video was incredible"
[Sentiment]: Positive
###


In [33]:

results = few_shot_pipeline(
    """For each food, suggest a good drink pairing:

[food]: tapas
[drink]: wine
###
[food]: pizza
[drink]: soda
###
[food]: jalapenos poppers
[drink]: beer
###
[food]: scone
[drink]:""",
    eos_token_id=eos_token_id,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each food, suggest a good drink pairing:

[food]: tapas
[drink]: wine
###
[food]: pizza
[drink]: soda
###
[food]: jalapenos poppers
[drink]: beer
###
[food]: scone
[drink]: tea
###


In [34]:

results = few_shot_pipeline(
    """Given a word describing how someone is feeling, suggest a description of that person.  The description should not include the original word.

[word]: happy
[description]: smiling, laughing, clapping
###
[word]: nervous
[description]: glancing around quickly, sweating, fidgeting
###
[word]: sleepy
[description]: heavy-lidded, slumping, rubbing eyes
###
[word]: confused
[description]:""",
    eos_token_id=eos_token_id,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


Given a word describing how someone is feeling, suggest a description of that person.  The description should not include the original word.

[word]: happy
[description]: smiling, laughing, clapping
###
[word]: nervous
[description]: glancing around quickly, sweating, fidgeting
###
[word]: sleepy
[description]: heavy-lidded, slumping, rubbing eyes
###
[word]: confused
[description]: looking around in confusion, scratching
###


In [36]:
#  overriding max_new_tokens to generate longer answers.

results = few_shot_pipeline(
    """Generate a book summary from the title:

[book title]: "Stranger in a Strange Land"
[book description]: "This novel tells the story of Valentine Michael Smith, a human who comes to Earth in early adulthood after being born on the planet Mars and raised by Martians, and explores his interaction with and eventual transformation of Terran culture."
###
[book title]: "The Adventures of Tom Sawyer"
[book description]: "This novel is about a boy growing up along the Mississippi River. It is set in the 1840s in the town of St. Petersburg, which is based on Hannibal, Missouri, where Twain lived as a boy. In the novel, Tom Sawyer has several adventures, often with his friend Huckleberry Finn."
###
[book title]: "Dune"
[book description]: "This novel is set in the distant future amidst a feudal interstellar society in which various noble houses control planetary fiefs. It tells the story of young Paul Atreides, whose family accepts the stewardship of the planet Arrakis. While the planet is an inhospitable and sparsely populated desert wasteland, it is the only source of melange, or spice, a drug that extends life and enhances mental abilities.  The story explores the multilayered interactions of politics, religion, ecology, technology, and human emotion, as the factions of the empire confront each other in a struggle for the control of Arrakis and its spice."
###
[book title]: "The Harry Potter"
[book description]:""",
    eos_token_id=eos_token_id,
    max_new_tokens=50,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


Generate a book summary from the title:

[book title]: "Stranger in a Strange Land"
[book description]: "This novel tells the story of Valentine Michael Smith, a human who comes to Earth in early adulthood after being born on the planet Mars and raised by Martians, and explores his interaction with and eventual transformation of Terran culture."
###
[book title]: "The Adventures of Tom Sawyer"
[book description]: "This novel is about a boy growing up along the Mississippi River. It is set in the 1840s in the town of St. Petersburg, which is based on Hannibal, Missouri, where Twain lived as a boy. In the novel, Tom Sawyer has several adventures, often with his friend Huckleberry Finn."
###
[book title]: "Dune"
[book description]: "This novel is set in the distant future amidst a feudal interstellar society in which various noble houses control planetary fiefs. It tells the story of young Paul Atreides, whose family accepts the stewardship of the planet Arrakis. While the planet is an in



# 4. Hugging Face APIs


 * Search and sampling to generate text
 * Auto* loaders for tokenizers and models
 * Model-specific loaders



In [38]:
display(cnn_sample.to_pandas())

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a
5,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili...",a1ebb8bb4d370a1fdf28769206d572be60642d70
6,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin...",7c0e61ac829a3b3b653e2e3e7536cc4881d1f264
7,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...,f0d73bdab711763e745cdc75850861c9018f235d
8,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can...",5e22bbfc7232418b8d2dd646b952e404df5bd048
9,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...,613d6311ec2c1985bd44707d1796d275452fe156


### Search and sampling in inference


In [40]:

# greedy search.
summarizer(cnn_sample["article"][0])

[{'summary_text': 'the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties . he will be able to gamble in a casino, buy a drink or see the horror film "Hostel: Part II" his agent and publicist had no comment on his plans . his latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix"'}]

In [43]:
#  beam search by specifying num_beams.

summarizer(cnn_sample["article"][0], num_beams=10)

[{'summary_text': 'Daniel Radcliffe says he has no plans to fritter his cash away on fast cars, drink and celebrity parties . at 18, he will be able to buy a drink in a pub or see the horror film "Hostel: Part II" his agent and publicist had no comment on his plans . "i\'ll definitely have some sort of party," he says .'}]

In [44]:
#  use sampling.
summarizer(cnn_sample["article"][0], do_sample=True)

[{'summary_text': 'he says he has no plans to fritter his cash away on fast cars, drink and celebrity parties . at 18, he will be able to buy a drink in a pub or see the horror film "Hostel: Part II" his agent and publicist had no comment on his plans . "i\'ll definitely have some sort of party," he said last month .'}]

In [45]:
#  modify sampling to be more greedy by limiting sampling to the top_k or top_p most likely next tokens.
summarizer(cnn_sample["article"][0], do_sample=True, top_k=10, top_p=0.8)

[{'summary_text': 'the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties . he will be able to gamble in a casino, buy a drink or see the horror film "Hostel: Part II" despite his growing fame and riches, he is keeping his feet on the ground .'}]

 ### Auto* loaders for tokenizers and models




In [52]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")


articles = list(map(lambda article: "summarize: " + article, cnn_sample["article"]))
display(pd.DataFrame(articles, columns=["prompts"]))

,prompts
0,"summarize: LONDON, England (Reuters) -- Harry ..."
1,summarize: Editor's note: In our Behind the Sc...
2,"summarize: MINNEAPOLIS, Minnesota (CNN) -- Dri..."
3,summarize: WASHINGTON (CNN) -- Doctors removed...
4,summarize: (CNN) -- The National Football Lea...
5,"summarize: BAGHDAD, Iraq (CNN) -- Dressed in a..."
6,"summarize: BAGHDAD, Iraq (CNN) -- The women ar..."
7,"summarize: BOGOTA, Colombia (CNN) -- A key reb..."
8,summarize: WASHINGTON (CNN) -- White House pre...
9,summarize: (CNN) -- Police and FBI agents are ...


In [48]:
# Tokenize the input
inputs = tokenizer(
    articles, max_length=1024, return_tensors="pt", padding=True, truncation=True
)
print("input_ids:")
print(inputs["input_ids"])
print("attention_mask:")
print(inputs["attention_mask"])

input_ids:
tensor([[21603,    10,   301,  ...,     0,     0,     0],
        [21603,    10, 11953,  ...,     0,     0,     0],
        [21603,    10,     3,  ...,     0,     0,     0],
        ...,
        [21603,    10, 10842,  ...,     0,     0,     0],
        [21603,    10,   549,  ...,     0,     0,     0],
        [21603,    10,    41,  ...,     0,     0,     0]])
attention_mask:
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [49]:
# Generate summaries
summary_ids = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    num_beams=2,
    min_length=0,
    max_length=40,
)
print(summary_ids)

tensor([[    0,   126,    10,    96,    23,   278,    31,    17,   317,    27,
            31,   195,    36,  1989, 28887,   976,     3,    88,   845,     3,
             5,   126,    10,    96,    23,    31,   195,  1728,    43,   128,
          1843,    13,  1088,   976,     3,    88,   845,     3,     5,   126],
        [    0,    16, 11171,    28,   167,  5274,  2550, 21154,    33,     3,
         14736,    15,  4094,   552,    79,    31,    60,  1065,    12,  2385,
            16,  1614,     3,     5,  5191,   845,    79,   557,   522,  2672,
          3991,    42, 12710,  3991,     3,     5,    96,  4188,   174,    12],
        [    0,   244,  7003,   127,    10,    96,    23,  1077,   141,     3,
             9, 12471,     6,  3097,    18,  6259,   339,  1590,     5,   275,
           132,    31,     7,  2948,    16,     8,   387,     6,   132,    31,
             7,  2948,    30,  1472,   121,     3, 14903,    13,   151,   130],
        [    0,   126,    10,     3,     7, 18461

In [53]:
# Decode the generated summaries
decoded_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
display(pd.DataFrame(decoded_summaries, columns=["decoded_summaries"]))

,decoded_summaries
0,"new: ""i don't think I'll be particularly extra..."
1,inmates with most severe mental illnesses are ...
2,"survivor: ""i probably had a 30-, 35-foot free ..."
3,new: spokesman: none appeared worrisome. new: ...
4,"new: ""your admitted conduct was not only illeg..."
5,new: youssif's parents say they know it's goin...
6,"women are driven to prostitution to ""save my c..."
7,fugitive from drug trafficking indictment kill...
8,"new: president bush says he will ""sadly accept..."
9,the 20-year-old AT4 anti-tank rocket launcher ...
